In [ ]:
%%capture
!pip install tts
!pip install fastapi uvicorn
!pip install pyngrok
!pip install python-multipart
!pip install nest_asyncio
!pip uninstall numpy -y
!pip install --upgrade pip setuptools wheel
!pip install TTS


In [ ]:
!pip install numpy==1.23.5


  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy>=1.24.3; python_version > "3.10", but you have numpy 1.23.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1

In [ ]:
from fastapi import FastAPI, Request, HTTPException, Query,Form
from fastapi.responses import StreamingResponse
from TTS.api import TTS
import uvicorn
import os
import uuid
from pyngrok import ngrok
import nest_asyncio

nest_asyncio.apply()

app = FastAPI()
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "output.mpeg")

device = "cuda"
tts = TTS(model_name="tts_models/en/ljspeech/glow-tts").to(device)

def audio_streamer(file_path: str, chunk_size: int = 1024):
    with open(file_path, "rb") as audio_file:
        while chunk := audio_file.read(chunk_size):
            yield chunk

@app.post("/speak")
async def speak(
    request: Request,
    model: str = Query("tts_models/en/ljspeech/glow-tts", description="text to speech"),
    encoding: str = Query("linear16", description="Audio encoding format"),
    container: str = Query("wav", description="Output container format"),
    sample_rate: int = Query(24000, description="Output audio sample rate in Hz")
):
    try:
        body = await request.json()
    except Exception:
        raise HTTPException(status_code=400, detail="Invalid JSON payload")

    if "text" not in body:
        raise HTTPException(status_code=400, detail="Missing 'text' in request body")

    text = body["text"]
    tts.tts_to_file(text=text, file_path=OUTPUT_PATH)
    response_headers = {
        "content-type": "audio/mpeg",
        "kipps-model-name": model,
        "kipps-model-uuid": str(uuid.uuid4()),
        "kipps-char-count": str(len(text)),
        "kipps-request-id": str(uuid.uuid4())
    }
    return StreamingResponse(audio_streamer(OUTPUT_PATH), media_type="audio/mpeg", headers=response_headers)

ngrok.set_auth_token("2v8WxF5ZeHTZz8xCPZoDdKkyNbx_G3XWmHLmGSdb7FiYhYWx")
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=5000)


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

 > tts_models/en/ljspeech/glow-tts is already downloaded.
 > vocoder_models/en/ljspeech/multiband-melgan is already downloaded.
 > Using model: glow_tts
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Vocoder Model: multiband_melgan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resam

INFO:     Started server process [5287]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     2401:4900:1c64:a894:b0db:a548:f453:37e7:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2401:4900:1c64:a894:b0db:a548:f453:37e7:0 - "GET /openapi.json HTTP/1.1" 200 OK
 > Text splitted to sentences.
['hello my name is harshita']
 > Processing time: 2.0672378540039062
 > Real-time factor: 0.8683060553334756
INFO:     54.86.50.139:0 - "POST /speak HTTP/1.1" 200 OK
 > Text splitted to sentences.
['hello my name is harshita, you are bad girl']
 > Processing time: 0.2095654010772705
 > Real-time factor: 0.05292419247931344
INFO:     54.86.50.139:0 - "POST /speak HTTP/1.1" 200 OK
